# Objective

Given a review , we have to determine whether the review is positive (4 or 5) or negative(1 or 2) by applying different models of machine learning and we can see the best result out of different classification models to classify the reviews.

# Dataset

The dataset is available on the kaggle https://www.kaggle.com/snap/amazon-fine-food-reviews/home. This dataset consists of 568,454 reviews of fine food from Amazon.This dataset includes the reviews from Oct 1999 to Oct 2012.The dataset is avaliable in two forms

1.csv file <br />
2.SQLite Databas

# Importing the Required Libraries 

In [2]:
import numpy as np 
import pandas as pd
# import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import re

# Importing the Data

In [3]:
pwd

'C:\\Users\\Vivek'

In [4]:
cd E:\Amazon Fine Food Reviews

E:\Amazon Fine Food Reviews


In [5]:
pwd

'E:\\Amazon Fine Food Reviews'

In [5]:
df = pd.read_csv('Reviews.csv')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
df.shape

(568454, 10)

### Creating New Feature named as Sentiment
Score = 3 Sentiment = 0 (Neutral) <br/>
Score > 3 Sentiment = 1 (Positive) <br/>
Score < 3 Sentiment = -1 (Negative) <br/>

In [7]:
df['Sentiment'] = df["Score"].apply(lambda x: 1 if x > 3 else (0 if x == 3 else -1))

In [8]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,-1
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,-1
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1


# Data Cleaning 

This stage is very important and quite time taking stage. You have to understand own your own what cleaning , you should do in our dataset and what type of cleaning is required according to our problem.Here, you will observe the duplicates in this dataset and you need to remove those duplicates in order to get the unbiased result for the analysis of data.One more important thing we need to focus on is that this dataset is based on timestamp so we need to sort our data according to the timestamp so that our model could perform very well on future unseen data (new reviews), for example, to deploy sentiment analysis of reviews in the system , each day there will be new reviews added for the different products which will act as new data for the system’s model so in order to perform well on new data with same results as it is preforming on old data, you need to sort them acc. to the timestamp.

Helpfulness - It is the ratio of people find this review helpful to total reaction on this review 
<br/>
So in this Numerator should always be less than or equals to Denominator. But we found some review where Numerator is greater than Denominator.

In [9]:
df[df.HelpfulnessNumerator > df.HelpfulnessDenominator]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
44736,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...,1
64421,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...,1


In [10]:
df = df[df.HelpfulnessNumerator <= df.HelpfulnessDenominator]

In [11]:
df[df['UserId'] == 'A395BORC6FGVXV']

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,-1
136303,136304,B002Y7526Y,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,-1
544172,544173,B000U9WZ54,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,-1


In [12]:
filtered_df = df.drop_duplicates(subset = {'UserId','ProfileName','Time'} ,keep = 'first', inplace = False)

In [13]:
final = filtered_df.sort_values('Time',axis= 0,inplace = False , na_position = 'last',ascending = True)

In [14]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
150523,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,5,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,1
150500,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,5,940809600,This whole series is great way to spend time w...,I can remember seeing the show when it aired o...,1
451855,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,5,944092800,Entertainingl Funny!,Beetlejuice is a well written movie ..... ever...,1
230284,230285,B00004RYGX,A344SMIA5JECGM,Vincent P. Ross,1,2,5,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta...",1
451854,451855,B00004CXX9,AJH6LUC1UT1ON,The Phantom of the Opera,0,0,5,946857600,FANTASTIC!,Beetlejuice is an excellent and funny movie. K...,1


In [15]:
final.shape

(352784, 11)

In [16]:
print ("Data remained after data cleaning is {} %.".format((final.shape[0]/df.shape[0])*100))

Data remained after data cleaning is 62.06047300387719 %.


In [17]:
final['Sentiment'].value_counts()

 1    273904
-1     52692
 0     26188
Name: Sentiment, dtype: int64

In [18]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [19]:
import codecs
import unidecode
import re
import spacy
nlp = spacy.load('en')

def spacy_cleaner(text):
    try:
        decoded = unidecode.unidecode(codecs.decode(text, 'unicode_escape'))
    except:
        decoded = unidecode.unidecode(text)
    apostrophe_handled = re.sub("’", "'", decoded)
    expanded = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in apostrophe_handled.split(" ")])
    parsed = nlp(expanded)
    final_tokens = []
    for t in parsed:
        if t.is_punct or t.is_space or t.like_num or t.like_url or str(t).startswith('@'):
            pass
        else:
            if t.lemma_ == '-PRON-':
                final_tokens.append(str(t))
            else:
                sc_removed = re.sub("[^a-zA-Z]", '', str(t.lemma_))
                if len(sc_removed) > 1:
                    final_tokens.append(sc_removed)
    joined = ' '.join(final_tokens)
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', joined)
    return spell_corrected

In [20]:
pd.set_option('display.max_colwidth', None)

In [21]:
final.Text[:10]

150523                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the 

In [22]:
[spacy_cleaner(t) for t in final.Text[:10]]

['this witty little book make my son laugh at loud recite it in the car as we be drive along and he always can sing the refrain he be learn about whale India droop rose love all the new word this book introduce and the silliness of it all this be classic book be willing to bet my son will still be able to recite from memory when he be in college',
 'I can remember see the show when it air on television year ago when I be child My sister later buy me the LP which I have to this day I be somethingI use this series of books amp song when I do my student teaching for preschooler amp turn the whole school on to it I be now purchase it on CD along with the book for my child amp the tradition live on',
 'Beetlejuice be well write movie everything about it be excellent from the acting to the special effect you will be delighted you choose to view this movie',
 'twist of rumplestiskin capture on film star michael keaton and geena davis in their prime Tim Burton masterpiece rumble with absurdity

In [23]:
final['clean_text'] = [spacy_cleaner(t) for t in final.Text]

C:\Users\Vivek\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: invalid escape sequence '\c'
  if __name__ == '__main__':
C:\Users\Vivek\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: invalid escape sequence '\<'
  if __name__ == '__main__':
C:\Users\Vivek\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: invalid escape sequence '\l'
  if __name__ == '__main__':
C:\Users\Vivek\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: invalid escape sequence '\i'
  if __name__ == '__main__':
C:\Users\Vivek\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: invalid escape sequence '\y'
  if __name__ == '__main__':
C:\Users\Vivek\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: invalid escape sequence '\T'
  if __name__ == '__main__':
C:\Users\Vivek\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: invalid escape sequence '\:'
  if __

In [24]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment,clean_text
150523,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,5,939340800,EVERY book is educational,"this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses: i love all the new words this book introduces and the silliness of it all. this is a classic book i am willing to bet my son will STILL be able to recite from memory when he is in college",1,this witty little book make my son laugh at loud recite it in the car as we be drive along and he always can sing the refrain he be learn about whale India droop rose love all the new word this book introduce and the silliness of it all this be classic book be willing to bet my son will still be able to recite from memory when he be in college
150500,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,5,940809600,This whole series is great way to spend time with your child,"I can remember seeing the show when it aired on television years ago, when I was a child. My sister later bought me the LP (which I have to this day, I'm thirty something).I used this series of books &amp; songs when I did my student teaching for preschoolers &amp; turned the whole school on to it. I am now purchasing it on CD, along with the books for my children 5 &amp; 2. The tradition lives on!",1,I can remember see the show when it air on television year ago when I be child My sister later buy me the LP which I have to this day I be somethingI use this series of books amp song when I do my student teaching for preschooler amp turn the whole school on to it I be now purchase it on CD along with the book for my child amp the tradition live on
451855,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,5,944092800,Entertainingl Funny!,Beetlejuice is a well written movie ..... everything about it is excellent! From the acting to the special effects you will be delighted you chose to view this movie.,1,Beetlejuice be well write movie everything about it be excellent from the acting to the special effect you will be delighted you choose to view this movie
230284,230285,B00004RYGX,A344SMIA5JECGM,Vincent P. Ross,1,2,5,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, starring michael keaton and geena davis in their prime. Tim Burton's masterpiece, rumbles with absurdity, and is wonderfully paced to the point where there is not a dull moment.",1,twist of rumplestiskin capture on film star michael keaton and geena davis in their prime Tim Burton masterpiece rumble with absurdity and be wonderfully pace to the point where there be not dull moment
451854,451855,B00004CXX9,AJH6LUC1UT1ON,The Phantom of the Opera,0,0,5,946857600,FANTASTIC!,"Beetlejuice is an excellent and funny movie. Keaton is hilarious as the wacky beetlejuice. The great special effects help the film. I think this is one of the best movies ever made and I'm sure you'll agree. For a good time, watch beetlejuice!",1,Beetlejuice be an excellent and funny movie Keaton be hilarious as the wacky beetlejuice the great special effect help the film I think this be of the good movie ever make and I be sure you will agree for good time watch beetlejuice


In [26]:
# final.to_csv('Reviews_Lemmatized.csv')

In [6]:
df = pd.read_csv("Reviews_Lemmatized.csv")
df.head()

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment,clean_text
0,150523,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,5,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,1,this witty little book make my son laugh at lo...
1,150500,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,5,940809600,This whole series is great way to spend time w...,I can remember seeing the show when it aired o...,1,I can remember see the show when it air on tel...
2,451855,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,5,944092800,Entertainingl Funny!,Beetlejuice is a well written movie ..... ever...,1,Beetlejuice be well write movie everything abo...
3,230284,230285,B00004RYGX,A344SMIA5JECGM,Vincent P. Ross,1,2,5,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta...",1,twist of rumplestiskin capture on film star mi...
4,451854,451855,B00004CXX9,AJH6LUC1UT1ON,The Phantom of the Opera,0,0,5,946857600,FANTASTIC!,Beetlejuice is an excellent and funny movie. K...,1,Beetlejuice be an excellent and funny movie Ke...


In [18]:
df[df['clean_text'] == None]

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment,clean_text


# Original Imbalanced Dataset

In [19]:
sent1 = "I love dogs"
sent2 = "I don't like dogs"
sent3 = "I adore cats"
sent4 = "I hate spiders"
sent5 = "I like dogs"
testing_text = pd.Series([sent1, sent2, sent3, sent4, sent5])
testing_target = pd.Series([1,0,1,0,1])

In [20]:
tv = TfidfVectorizer(stop_words=None, max_features=100000)
testing_tfidf = tv.fit_transform(testing_text)

In [24]:
pd.DataFrame(testing_tfidf)

,0
0,"(0, 2)\t0.5564505207186616\n (0, 6)\t0.8308..."
1,"(0, 5)\t0.5568161504458247\n (0, 3)\t0.6901..."
2,"(0, 1)\t0.7071067811865475\n (0, 0)\t0.7071..."
3,"(0, 7)\t0.7071067811865475\n (0, 4)\t0.7071..."
4,"(0, 5)\t0.7694470729725092\n (0, 2)\t0.6387..."


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tvec = TfidfVectorizer(stop_words=None, max_features=100000, ngram_range=(1, 3))
lr = LogisticRegression()

In [16]:
tvec

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0,
                max_features=100000, min_df=1, ngram_range=(1, 3), norm='l2',
                preprocessor=None, smooth_idf=True, stop_words=None,
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score

def lr_cv(splits, X, Y, pipeline, average_method):
    
    kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=777)
    accuracy = []
    precision = []
    recall = []
    f1 = []
    for train, test in kfold.split(X, Y):
        lr_fit = pipeline.fit(X[train], Y[train])
        prediction = lr_fit.predict(X[test])
        scores = lr_fit.score(X[test],Y[test])
        
        accuracy.append(scores * 100)
        precision.append(precision_score(Y[test], prediction, average=average_method)*100)
        print('              negative    neutral     positive')
        print('precision:',precision_score(Y[test], prediction, average=None))
        recall.append(recall_score(Y[test], prediction, average=average_method)*100)
        print('recall:   ',recall_score(Y[test], prediction, average=None))
        f1.append(f1_score(Y[test], prediction, average=average_method)*100)
        print('f1 score: ',f1_score(Y[test], prediction, average=None))
        print('-'*50)

    print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
    print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
    print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
    print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))

In [9]:
from sklearn.pipeline import Pipeline

original_pipeline = Pipeline([
    ('vectorizer', tvec),
    ('classifier', lr)
])

In [10]:
lr_cv(5, df.clean_text, df.Sentiment, original_pipeline, 'macro')

ValueError: np.nan is an invalid document, expected byte or unicode string.

# Text Preprocessing

Data preprocessing is also an important process in which you transform our data before applying the different models on dataset so that you can achieve the better results. In this stage , you can perform certain steps on our reviews(text) as mentioned below:-

* Begin with defining your own function which will remove the html tags or we can use Beautiful Soup .
* Removal of punctuations or limited set of special characters like . or “,” or removing whitespaces or etc from the reviews.
* Check if the word is made up of english letters and is not alpha-numeric.
* Check if the length of word is greater than 2 (as there is no adjective in 2 letters).
* uppercase to lower case conversion.
* Removal of Stopwords(commonly used word such as “a”, “the” etc to be ignored).
* Stemming(Snowball Stemming) the word of each reviews.

In [ ]:
def cleanhtml(sent):# function for cleaning html tags
    cleanr = re.compile('<.*?>')
    cleaned = re.sub(cleanr,' ',sent)
    return cleaned

In [ ]:
def cleanpunc(sent):# function for cleaning punctuations
    clean = re.sub(r'[?|!|$|#|\'|"|:]',r'',sent)
    clean = re.sub(r'[,|(|)|.|\|/]',r' ',clean)
    return clean

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
stop = set(stopwords.words("english"))


In [ ]:
from nltk.stem.snowball import SnowballStemmer
st = SnowballStemmer("english")
st.stem('burned')

In [ ]:
i=0
all_positive_reviews = []
all_negative_reviews = []
all_neutral_reviews = []

final_string = []
stem_data = " "
for p in final['Text'].values:
    filtered_sens = []#filtered word
    p = cleanhtml(p)
    for w in p.split():
       # print(w)
        punc = cleanpunc(w)
        for s in punc.split():
            #print(w)
            if (s.isalpha()) & (len(s)>2):
                if s.lower() not in stop:
                    stem_data = (st.stem(s.lower())).encode('utf8')
                    filtered_sens.append(stem_data)
                    if (final['Sentiment'].values)[i] == 1:
                        all_positive_reviews.append(stem_data)
                    if (final['Sentiment'].values)[i] == -1:
                        all_negative_reviews.append(stem_data)
                    if (final['Sentiment'].values)[i] == 0:
                        all_negative_reviews.append(stem_data)    
                else:
                    continue
            else:
                continue
    #print(filtered_sens)
    str1 = b" ".join(filtered_sens)
    #print(str1)
    final_string.append(str1)
    i+=1

In [ ]:
final['CleanedText'] = final_string
final.head()

In [ ]:
final.CleanedText[0]